<a href="https://colab.research.google.com/github/lanyu0322/phd_semester1/blob/master/foot_traffic_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -- mount google drive
from google.colab import drive
drive.mount("/content/drive")

In [0]:
# -- install geopandas
!pip install geopandas

     |████████████████████████████████| 921kB 5.2MB/s 
     |████████████████████████████████| 11.8MB 52.3MB/s 
     |████████████████████████████████| 10.1MB 41.8MB/s 


In [0]:
import os
import time
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
from scipy.optimize import leastsq

plt.rcParams["figure.dpi"] = 150

In [0]:
# -- set the root directory for the project                                     
rpath = os.path.join("drive", "My Drive", "lwir")

In [0]:
# -- some utility functions

def convert_to_nyc(data):
    if (data.geometry.crs["init"] != "epsg:2263"):
        data.geometry = data.geometry.to_crs(epsg=2263)



def fit_3_gaussians(data, guess):
    """ Fit a 3 Gaussian model to a time series. """

    # -- define a 1D gaussian
    def gauss(xarr, x0, sig, scl):
        amp = scl / (sig * np.sqrt(2.0 * np.pi))
        val = -0.5 * ((xarr - x0) / sig)**2
  
        return amp * np.exp(val)

    # -- define 3 gaussian model
    def gauss3(xval, param):
        m1, m2, m3, sd1, sd2, sd3, scl1, scl2, scl3, off = param

        model = gauss(xval, m1, sd1, scl1) + gauss(xval, m2, sd2, scl2) + \
            gauss(xval, m3, sd3, scl3) + off

        return model

    # -- define model error
    def res(param, xval, yval):

        return yval - gauss3(xval, param)

    # -- do the fit
    xval = np.arange(len(data))
    sol = leastsq(res, guess, args=(xval, data))

    # -- sort the solution
    gpar       = sol[0][:9].reshape(3, 3)
    gpar       = gpar[:, np.argsort(gpar[0])]
    sol[0][:9] = gpar.flatten()

    return list(sol) + [gauss3(xval, sol[0])]


def read_lehd():
    """ Read in the LEHD LODES data, joining with the census tract shapes. """

    # -- set the file names
    lpath = os.path.join(rpath, "data", "lehd")
    wname = os.path.join(lpath, "ny_wac_S000_JT00_2017.csv") 
    rname = os.path.join(lpath, "ny_rac_S000_JT00_2017.csv")

    # -- read in the data for NY state
    print("READ_LEHD: loading the worker and resident totals for NY state...")
    wdata = pd.read_csv(wname)
    rdata = pd.read_csv(rname)

    # -- rename the geocode for merging with the shape data
    wdata.rename(columns = {'w_geocode': 'GEOID10'}, inplace=True)
    rdata.rename(columns = {'h_geocode': 'GEOID10'}, inplace=True)

    # -- merge the worker and residents
    cols = ["GEOID10", "C000"]
    lehd = pd.merge(wdata[cols], rdata[cols], on="GEOID10", 
                    suffixes=["_w", "_r"], how="outer").fillna(0)

    # -- set the total number of workers and residents
    lehd["total_p"] = lehd["C000_w"] + lehd["C000_r"]

    # -- load the shapes
    print("READ_LEHD: loading the census blocks for NY state...")
    t0      = time.time()
    cpath   = os.path.join("drive", "My Drive", "lwir", "data", "lehd", 
                           "tl_2019_36_tabblock10")
    cname   = os.path.join(cpath, "tl_2019_36_tabblock10.shp")
    bl_full = gp.read_file(cname)
    print("READ_LEHD:   elapsed time = {0}s" \
              .format(round(time.time() - t0), 2))

    # -- convert the shape geo ID to integer
    bl_full["GEOID10"] = bl_full["GEOID10"].astype(int)

    # -- sub-select only New York City Counties
    print("READ_LEHD: sub-selecting only Manhattan...")
    bl_mn = bl_full[bl_full["COUNTYFP10"] == "061"]

    # -- merge LEHD data and geographic data
    print("READ_LEHD: merging Manhattan census blocks with LODES...")
    lehd_bl = bl_mn.merge(lehd, on="GEOID10")

    return lehd_bl

In [0]:
# -- some useful plotting functions

def plot_camera_locations(boro, parks, camdat, cambad=None, write=True):

    fig, ax = plt.subplots(figsize=[4, 6])
    fig.subplots_adjust(0.2, 0.1, 0.9, 0.95)
    ax.set_facecolor("lightblue")
    boro.plot(ax=ax, color="#666666")
    parks.plot(ax=ax, color="orange", alpha=0.5)
    camdat.plot("lon", "lat", ax=ax, color="lime", kind="scatter", s=5, label='Camera Locations')
    if cambad is not None:
        cambad.plot.scatter("lon", "lat", ax=ax, color="red", s=20,  
                            marker="x", label='Dropped cameras')
    ax.legend(loc='upper left')
    

    if write:
        oname = os.path.join(rpath, "output", "camera_locations.png")
        fig.savefig(oname, clobber=True)

    return



def plot_paramter_map(boro, camdat, col, write=True):

    # -- set colormap range
    vlo = camdat[col].quantile(0.2)
    vhi = camdat[col].quantile(0.8)

    # -- set the x and y ticks
    xticks = np.arange(-74.04, -73.92, 0.04)
    yticks = np.arange(40.675, 40.875, 0.025)

    # -- create the figure
    fig, ax = plt.subplots(figsize=[7, 6])
    fig.subplots_adjust(0.2, 0.1, 0.9, 0.95)
    ax.set_facecolor("lightblue")

    # -- bug in either pandas or geopandas... must scatter plot first
    camdat.plot("lon", "lat", ax=ax, c=col, kind="scatter", s=15, 
                cmap="viridis", vmin=vlo, vmax=vhi, xticks=xticks, 
                yticks=yticks)
    boro.plot(ax=ax, color="#666666")
    
  

    
# Lan add the park prop
    
# def plot_paramter_map_lan(boro, camdat, col, write=True):

#     # -- set colormap range
#     vlo = camdat[col].quantile(0.2)
#     vhi = camdat[col].quantile(0.8)

#     # -- set the x and y ticks
#     xticks = np.arange(-74.04, -73.92, 0.04)
#     yticks = np.arange(40.675, 40.875, 0.025)

#     # -- create the figure
#     fig, ax = plt.subplots(figsize=[7, 6])
#     fig.subplots_adjust(0.2, 0.1, 0.9, 0.95)
#     ax.set_facecolor("lightblue")

#     # -- bug in either pandas or geopandas... must scatter plot first
#     camdat.plot("lon", "lat", ax=ax, c=col, kind="scatter", s=15, 
#                 cmap="viridis", vmin=vlo, vmax=vhi, xticks=xticks, 
#                 yticks=yticks)
#     boro.plot(ax=ax, color="#666666")

    
    # -- replotting the points so that they are in front of the boro
    camdat.plot("lon", "lat", ax=ax, c=col, kind="scatter", s=15, 
                cmap="viridis", vmin=vlo, vmax=vhi, colorbar=False)
    
    

    
    

    # -- write to file if desired
    if write:
        oname = os.path.join(rpath, "output", col + "_map.png")
        fig.savefig(oname, clobber=True)

    return

In [0]:
# # -- read in the counts                                                         
# cpath  = os.path.join(rpath, "data", "nycdot")
# cname  = "cams_ft_wd.feather"
# camdat = pd.read_feather(os.path.join(cpath, cname))
# bind   = [5, 18, 19, 48, 51, 52, 66, 69, 72, 74, 79, 80, 81, 94, 95, 101, 102,
#           103, 105, 113, 114, 115, 151, 121, 62, 141]
# cambad = camdat[camdat.index.isin(bind)]
# camdat = camdat[~camdat.index.isin(bind)].reset_index(drop=True)
# camarr = camdat.drop(columns=["cam_id", "lat", "lon"]).values

# # -- read in the parks data                                                     
# ppath = os.path.join(rpath, "data", "parks", "properties")
# pname = "geo_export_5b605687-0f81-4fe4-9fb7-ec27cd43ab5f.shp"
# parks = gp.read_file(os.path.join(ppath, pname))
# parks = parks[parks["borough"] == "M"]

In [0]:

###  4 model
# -- read in the counts                                                         
cpath  = os.path.join(rpath, "data", "nycdot")
cname  = "cams_ft_wd.feather"
camdat = pd.read_feather(os.path.join(cpath, cname))
bind   = [0, 5, 7, 13, 18, 19, 48, 51, 52, 62, 69, 72, 79, 98, 99, 101, 102,
          104, 105, 109, 113, 114, 115, 151, 121, 125, 137, 141, 146, 151]
cambad = camdat[camdat.index.isin(bind)]
camdat = camdat[~camdat.index.isin(bind)].reset_index(drop=True)
camarr = camdat.drop(columns=["cam_id", "lat", "lon"]).values

# -- read in the parks data                                                     
ppath = os.path.join(rpath, "data", "parks", "properties")
pname = "geo_export_5b605687-0f81-4fe4-9fb7-ec27cd43ab5f.shp"
parks = gp.read_file(os.path.join(ppath, pname))
# parks = parks[parks["borough"] == "M"]


In [0]:
print(camdat)

     cam_id        lat        lon  ...        93        94        95
0       421  40.761268 -73.983564  ...  1.767610  1.668866  1.504888
1       514  40.770207 -73.986869  ...  0.040762  0.044282  0.055718
2       838  40.718427 -73.994830  ...  0.613099  0.580427  0.555230
3       910  40.786516 -73.952456  ...  0.269355  0.280303  0.228795
4      1028  40.752424 -74.000899  ...  1.081694  1.130596  1.001256
..      ...        ...        ...  ...       ...       ...       ...
123     725  40.728600 -74.005356  ...  1.229720  1.101271  1.045022
124     826  40.807061 -73.933681  ...  0.087243  0.119599  0.089725
125     941  40.761216 -73.957815  ...  0.120381  0.147947  0.132480
126     985  40.702389 -74.012806  ...  0.789338  0.739932  0.712539
127     403  40.744818 -73.977985  ...  0.817851  0.743646  0.721411

[128 rows x 99 columns]


In [0]:
# -- read in the LEHD data                                                      
try:
  lehd
except:
  lehd = read_lehd()

In [0]:
# -- read in the Manhattan boundaries                                           
bpath = os.path.join(rpath, "data", "boro_boundaries")
bname = "Borough Boundaries.geojson"
boro  = gp.read_file(os.path.join(bpath, bname))
boro  = boro[boro["boro_name"] == "Manhattan"]

# -- fit a 3-Gaussian model to the data                                         
avgs   = [35, 47, 68]
sigs   = [3, 3, 3]
scls   = [5, 5, 5]
off    = [0.5]
guess  = avgs + sigs + scls + off
ncam   = camarr.shape[0]
plsq   = [fit_3_gaussians(camarr[i], guess) for i in range(ncam)]
parr   = np.array([tplsq[0] for tplsq in plsq])


# -- add the parameters back to the camera dataframe                            
cnames = ["m1", "m2", "m3", "sig1", "sig2", "sig3", "scl1", "scl2", "scl3",
          "off"]
for ii in range(len(cnames)):
    camdat[cnames[ii]] = pd.Series(parr[:, ii])


# -- convert means and widths to hours                                          
for cname in ["m1", "m2", "m3", "sig1", "sig2", "sig3"]:
    camdat[cname] *= 0.25

# plot the parameters by Lan


In [0]:
camdat_len = len(camdat)
wrtot_manhattan = np.zeros(camdat_len)

for i in range(camdat_len):
  wrtot_manhattan[i] = integrate_geodata(lehd.geometry, lehd.total_p, camdat.lat[i], camdat.lon[i], 500.)
  

# Srat map and Heat map with legend

In [0]:
opath = os.path.join(rpath, "output")
cname = os.path.join(opath, "camdat_features.csv")
cdata = pd.read_csv(cname)
cdata.head()


camdat['srat'] = 'srat'

In [0]:
camdat.head()

In [0]:
# -- plot the parameters
fig = plt.figure()
plot_paramter_map(boro, cdata, 'srat')
#plt.title("Color-Coded Scatter Plot of Mean in Morning Commute", c = "grey")
outfile = os.path.join("drive", "My Drive", "lwir", "output", "srat.png")
fig.savefig(outfile)

In [0]:
import math 


C000_r_log = []
C000_w_log = []

for i in range(len(lehd)):
  C000_r_log.append(math.log10(lehd.C000_r[i]+1))
  C000_w_log.append(math.log10(lehd.C000_w[i]+1))
  

In [0]:
lehd['C000_r_log'] = C000_r_log
lehd['C000_w_log'] = C000_w_log

In [0]:
#heat map
# -- plot it

fig = plt.figure()


  # -- set the x and y ticks
xticks = np.arange(-74.04, -73.92, 0.04)
yticks = np.arange(40.675, 40.875, 0.025)

xticks=xticks
yticks=yticks
    # -- create the figure
fig, ax = plt.subplots(figsize=[7, 6])
fig.subplots_adjust(0.2, 0.1, 0.9, 0.95)
ax.set_facecolor("lightblue")


lehd.plot("C000_r_log", cmap="gist_heat", ax=ax, legend=True, vmin=0, vmax=4)

plt.title("Residents")



outfile = os.path.join("drive", "My Drive", "lwir", "output", "heat_map_residents.png")
fig.savefig(outfile)


In [0]:

import pyproj
from shapely.geometry import Point

# -- define helper function for integrating within a circle
def integrate_geodata(geo, vals, lat, lon, rad):
  
  # -- convert lat/lon to NY State Plane
  ll_nyspd = pyproj.Proj(init="epsg:2263", preserve_units=True)(lon, lat)
  
  # -- define a circle
  circ = Point(ll_nyspd[0], ll_nyspd[1]).buffer(rad)
 
  # -- calculate the intersection of the circle with the geometry
  inter = geo.intersection(circ)
  
  # -- determine the overlap fraction
  frac = inter.area / geo.area
  return (frac * vals).sum() 

def convert_to_nyc(data):
  if (data.geometry.crs["init"] != "epsg:2263"):
    data.geometry = data.geometry.to_crs(epsg=2263)

In [0]:
# for each camera:
camdat_len = len(camdat)

park_fname = os.path.join("drive", "My Drive", "lwir", "data", "parks", "properties", "geo_export_5b605687-0f81-4fe4-9fb7-ec27cd43ab5f.shp")
park_data = gp.read_file(park_fname)
print("init park epsg: ", park_data.geometry.crs)

convert_to_nyc(park_data)
print("converted park epsg: ", park_data.geometry.crs)

# -- get the parks area within a radius
park_area = np.array([integrate_geodata(park_data.geometry, park_data.area, camdat.lat[i], camdat.lon[i], 500.) for i in range(camdat_len)])
